In [44]:
class Person:
    def __init__(self,first_name,last_name,dob,height):
        self.first_name = first_name
        self.last_name = last_name
        self.dob = dob
        self.height = height
        
    def __repr__(self):
        return f'Person({self.first_name},{self.last_name},{self.dob},{self.height})'

In [45]:
from datetime import date

In [46]:
p1 = Person('Dean','Winchester',date(1975,5,1),182)

In [47]:
p1 

Person(Dean,Winchester,1975-05-01,182)

In [48]:
import os
os.system("pip install -U marshmallow")

0

In [49]:
from marshmallow import Schema,fields


In [51]:
class PersonSchema(Schema):
    first_name = fields.Str()
    last_name = fields.Str()
    dob = fields.Date()
    height = fields.Int()


In [52]:
person_schema = PersonSchema()

In [62]:
person_schema.dump(p1)

{'last_name': 'Winchester',
 'first_name': 'Dean',
 'height': 182,
 'dob': '1975-05-01'}

In [54]:
print(type(p1),p1)

<class '__main__.Person'> Person(Dean,Winchester,1975-05-01,182)


In [57]:
import json

In [71]:
data = person_schema.dump(p1)

In [72]:
data

{'last_name': 'Winchester',
 'first_name': 'Dean',
 'height': 182,
 'dob': '1975-05-01'}

In [74]:
datajson = json.dumps(data)

In [75]:
datajson

'{"last_name": "Winchester", "first_name": "Dean", "height": 182, "dob": "1975-05-01"}'

In [76]:
jso = json.loads(datajson)

In [77]:
jso

{'last_name': 'Winchester',
 'first_name': 'Dean',
 'height': 182,
 'dob': '1975-05-01'}

In [79]:
jso['last_name']

'Winchester'

In [80]:
from collections import namedtuple

In [81]:
PT = namedtuple('PT','first_name,last_name,dob,height')

In [83]:
p2 = PT('Sam','Winchester',date(1982,11,13),192)

In [84]:
p2

PT(first_name='Sam', last_name='Winchester', dob=datetime.date(1982, 11, 13), height=192)

In [85]:
person_schema.dumps(p2)

'{"last_name": "Winchester", "first_name": "Sam", "height": 192, "dob": "1982-11-13"}'

In [86]:
data


{'last_name': 'Winchester',
 'first_name': 'Dean',
 'height': 182,
 'dob': '1975-05-01'}

In [88]:
PT2 = namedtuple('PT2','first_name,last_name,age')
p3 = PT2('Cass','Heaven',2000)

In [91]:
data = person_schema.dumps(p3)

In [92]:
data

'{"last_name": "Heaven", "first_name": "Cass"}'

In [95]:
person_partial = PersonSchema(only=('first_name','last_name'))

In [96]:
person_schema.dumps(p1)

'{"last_name": "Winchester", "first_name": "Dean", "height": 182, "dob": "1975-05-01"}'

In [97]:
person_partial.dumps(p1)

'{"last_name": "Winchester", "first_name": "Dean"}'

In [100]:
person_partial = PersonSchema(exclude=['dob'])

In [110]:
person_partial.dumps(p1)

'{"last_name": "Winchester", "first_name": "Dean", "height": 182}'

In [111]:
class Movie:
    def __init__(self,title,year,actors):
        self.title = title
        self.year = year
        self.actors = actors

In [112]:
class MovieSchema(Schema):
    title = fields.Str()
    year = fields.Integer()
    actors = fields.Nested(PersonSchema,many = True)
    

In [113]:
p1,p2

(Person(Dean,Winchester,1975-05-01,182),
 PT(first_name='Sam', last_name='Winchester', dob=datetime.date(1982, 11, 13), height=192))

In [117]:
supernatural = Movie('Supernatural',2005,[p1,PT('Dean','Winchester',date(1975,5,1),185)])

In [118]:
supernatural

In [119]:
MovieSchema().dumps(supernatural)

'{"title": "Supernatural", "actors": [{"last_name": "Winchester", "first_name": "Dean", "height": 182, "dob": "1975-05-01"}, {"last_name": "Winchester", "first_name": "Dean", "height": 185, "dob": "1975-05-01"}], "year": 2005}'

In [120]:
class PersonSchema(Schema):
    first_name = fields.Str()
    last_name = fields.Str()
    dob = fields.Date()
    height = fields.Int()

In [121]:
person_schema = PersonSchema()

In [123]:
person_schema.load(dict(first_name = 'Dean',
                       last_name = 'Winchester',
                       dob = '1975-5-1',
                       height=182))

{'last_name': 'Winchester',
 'first_name': 'Dean',
 'height': 182,
 'dob': datetime.date(1975, 5, 1)}

In [124]:
from marshmallow import post_load


In [125]:
class PersonSchema(Schema):
    first_name = fields.Str()
    last_name = fields.Str()
    dob = fields.Date()
    height = fields.Int()
    
    @post_load
    def make_person(self,data):
        return Person(**data)

In [130]:
class MovieSchema(Schema):
    title = fields.Str()
    year = fields.Integer()
    actors = fields.Nested(PersonSchema,many = True)
    
    @post_load
    def make_movie(self,data):
        return Movie(**data)

In [134]:
movie_schema = MovieSchema()
person_schema = PersonSchema()

In [135]:
json_data='''
{
    "actors":[
    {"first_name":"Dean","last_name":"Winchester","dob":"1975-05-01","height":182},
    {"first_name":"Sam","last_name":"Winchester","dob":"1982-11-13","height":192}
    ],
    "title":"Supernatural",
    "year":2005
}'''